In [56]:
import numpy as np
import scipy as sp
import scipy.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline
import torch 
import torchvision

In [57]:
training = np.load('reg_multi.npy')

In [58]:
x_dataset = torch.from_numpy(training[:1000,0:16]).t()
y_dataset = torch.from_numpy(training[:1000,16]).t()
number_columns = 16
print(training.shape)
print(x_dataset.shape)
print(y_dataset.shape)

(60000, 17)
torch.Size([16, 1000])
torch.Size([1000])


In [59]:
# train_loader = torch.utils.data.DataLoader(t,batch_size=size,shuffle=True)

In [ ]:
# f = Ax + b
A = torch.randn((1,number_columns),requires_grad=True)
b = torch.randn(1,requires_grad=True)
def model(x_input):
    return A.mm(x_input) + b
def loss(y_predicted, y_target):
    return ((y_predicted - y_target)**2).sum()

In [60]:
optimizer = torch.optim.Adam([A,b], lr=0.001)
max_epoch = 10000
for epoch in range(max_epoch):
    optimizer.zero_grad()
    y_pred = model(x_dataset)
    c_loss = loss(y_pred,y_dataset)
    c_loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f"t = {epoch}, loss = {c_loss}")

t = 0, loss = 9304.427734375
t = 1000, loss = 2278.260009765625
t = 2000, loss = 706.2139892578125
t = 3000, loss = 378.134033203125
t = 4000, loss = 335.5292053222656
t = 5000, loss = 333.49249267578125
t = 6000, loss = 333.4761962890625
t = 7000, loss = 333.4761657714844
t = 8000, loss = 333.4761657714844
t = 9000, loss = 333.4761657714844


In [61]:
print(f"A = {A.detach().numpy()}, b = {b.item()}")
torch.save(model, "multi_regression.pt")

A = [[-0.00746233  0.06264612 -0.00745292 -0.02609568  0.00342379  0.01760814
  -0.0276236   0.01418392 -0.01270398  0.04081987 -0.05075576 -0.02595832
   0.00668264  0.09286415 -0.07409634  0.01621172]], b = 0.01607140153646469
